# Preprocessing


In [ ]:
import pandas as pd
import numpy as np
import warnings
import os
import time
from math import floor

from sklearn.preprocessing import StandardScaler
from utils import get_subburst_preserved_train_test, lee_liu_score

import optuna
from optuna.samplers import TPESampler

import ast
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from pulearn import (
    ElkanotoPuClassifier,
    WeightedElkanotoPuClassifier,
    BaggingPuClassifier,
)
from pu_modified_lr.mlr import ModifiedLogisticRegression
from PUExtraTrees.trees import PUExtraTrees
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import recall_score, fbeta_score
import shap

warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)

import matplotlib.pyplot as plt
import seaborn as sns
from umap import UMAP

plotpar = {"text.usetex": True}
plt.rcParams.update(plotpar)

from utils import (
    SUPERVISED_PAPER_REPEATERS,
    UNSUPERVISED_PAPER_REPEATERS,
    MORPHOLOGY_REPEATERS,
    PCC_REPEATER_CANDIDATES,
    REFERENCE_PAPER_REPEATERS,
)
from utils import custom_cmap

RANDOM_SEED = 42

In [ ]:
df = pd.read_csv("./data/features_extracted/combined_2021_23_catalog.csv")
pd.set_option("display.max_columns", None)
df.columns = df.columns.str.strip()

FEATURES = [
    "ra",
    "snr_fitb",
    "log_dm_exc_ne2001",
    "log_bc_width",
    "log_flux",
    "log_fluence",
    "sp_idx",
    "sp_run",
    "log_in_duration",
    "log_peak_freq",
    "log_fre_width",
    "log_T_B",
    "log_energy",
]
FEATURE_LABELS = [
    "Right Ascension",
    "SNR (fitburst)",
    "Extragalactic DM (NE2001)",
    "Boxcar width",
    "Flux",
    "Fluence",
    "Spectral index",
    "Spectral running",
    "Rest-frame width",
    "Peak frequency",
    "Frequency width",
    "Brightness temperature",
    "Burst energy",
]
X = df[FEATURES]
y = df["is_repeater"]

df

Read results from the previous notebook:

In [ ]:
supervised_models_df = pd.read_csv("data/supervised_models_optimised.csv")
supervised_models_df.sort_values("recall_mean", ascending=False)

In [ ]:
# Select models based on recall
BASE_CLASSIFIERS = list(
    supervised_models_df.sort_values("recall_mean", ascending=False)["model"][:3]
)
print(BASE_CLASSIFIERS)
# 2 of our base classifiers are considered to be well-calibrated: their probability can be interpreted as a confidence level
CALIBRATED_CLASSIFIERS = ["LogisticRegression", "LinearDiscriminantAnalysis"]

In [ ]:
# Make a standard train-test split that will be used for optimisation
np.random.seed(RANDOM_SEED)
X_train, X_val, y_train, y_val = get_subburst_preserved_train_test(
    original_df=df, X=X, y=y, test_size=0.2
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# PU Classifier Optimisation


In [ ]:
NUM_TRIALS = 150
optimised_models = []  # store optimised model objects
models_info = (
    []
)  # store information about each model, including optimal hyperparameters

In [ ]:
def best_base_classifier(clf_name):
    """
    Given a classifier name, find the optimal parameters saved in supervised_models_df.
    Then, find the corresponding class for the model in the global scope, and instantiate that class using the parameters
    """
    row = supervised_models_df[supervised_models_df["model"] == clf_name].iloc[0]
    params = ast.literal_eval(row["params"])
    base_clf = globals()[clf_name](**params)
    return base_clf

### Elkanoto


In [ ]:
np.random.seed(RANDOM_SEED)


def elkanoto_objective(trial):
    np.random.seed(RANDOM_SEED)
    base_clf_option = trial.suggest_categorical("base_clf_option", BASE_CLASSIFIERS)
    best_classifier = best_base_classifier(base_clf_option)

    # If necessary, calibrate the classifier using Platt scaling
    if base_clf_option not in CALIBRATED_CLASSIFIERS:
        best_classifier = CalibratedClassifierCV(
            best_classifier, method="sigmoid", cv=10
        )
    classifier_obj = ElkanotoPuClassifier(
        estimator=best_classifier,
        hold_out_ratio=trial.suggest_float("hold_out_ratio", 0.1, 0.8),
    )
    # Fit and evaluate on the same sets of data each time
    classifier_obj.fit(X_train_scaled, y_train)
    y_pred = classifier_obj.predict(X_val_scaled)

    # Maximise lee-liu score
    return lee_liu_score(y_known=y_val, y_pred=y_pred)


sampler = TPESampler(seed=RANDOM_SEED)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(elkanoto_objective, n_trials=NUM_TRIALS)

In [ ]:
# Load best params and save. Re-instantiate the classifeir using those parameters and save it to a list
best_params = study.best_params
best_clf_option = best_params["base_clf_option"]
best_classifier = best_base_classifier(best_clf_option)
if best_clf_option not in CALIBRATED_CLASSIFIERS:
    print("Calibating...")
    best_classifier = CalibratedClassifierCV(best_classifier, method="sigmoid", cv=10)
optimised_elkanoto = ElkanotoPuClassifier(
    estimator=best_classifier,
    hold_out_ratio=best_params["hold_out_ratio"],
    random_state=RANDOM_SEED,
)
optimised_models.append(optimised_elkanoto)
data = {
    "model": "Classic Elkanoto",
    "params": best_params,
    "ll_score": study.best_value,
}
models_info.append(data)
print(data)

### Weighted Elkanoto


In [ ]:
np.random.seed(RANDOM_SEED)


def weighted_elkanoto_objective(trial):
    np.random.seed(RANDOM_SEED)
    base_clf_option = trial.suggest_categorical("base_clf_option", BASE_CLASSIFIERS)
    best_classifier = best_base_classifier(base_clf_option)
    if base_clf_option not in CALIBRATED_CLASSIFIERS:
        best_classifier = CalibratedClassifierCV(
            best_classifier, method="sigmoid", cv=10
        )
    classifier_obj = WeightedElkanotoPuClassifier(
        estimator=best_classifier,
        hold_out_ratio=trial.suggest_float("hold_out_ratio", 0.1, 0.8),
        # Requires cardinality of labeled and unlabeled data
        labeled=sum(y_train == 1),
        unlabeled=sum(y_train == 0),
    )
    classifier_obj.fit(X_train_scaled, y_train)
    y_pred = classifier_obj.predict(X_val_scaled)
    return lee_liu_score(y_known=y_val, y_pred=y_pred)


sampler = TPESampler(seed=RANDOM_SEED)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(weighted_elkanoto_objective, n_trials=NUM_TRIALS)

In [ ]:
best_params = study.best_params
best_clf_option = best_params["base_clf_option"]
best_classifier = best_base_classifier(best_clf_option)
if best_clf_option not in CALIBRATED_CLASSIFIERS:
    print("Calibating...")
    best_classifier = CalibratedClassifierCV(best_classifier, method="sigmoid", cv=10)
optimised_welkanoto = WeightedElkanotoPuClassifier(
    estimator=best_classifier,
    hold_out_ratio=best_params["hold_out_ratio"],
    labeled=sum(y == 1),
    unlabeled=sum(y == 0),
    random_state=RANDOM_SEED,
)
optimised_models.append(optimised_welkanoto)
data = {
    "model": "Weighted Elkanoto",
    "params": best_params,
    "ll_score": study.best_value,
}
models_info.append(data)
print(data)

### Bagging


In [ ]:
np.random.seed(RANDOM_SEED)


def bagging_objective(trial):
    np.random.seed(RANDOM_SEED)
    base_clf_option = trial.suggest_categorical("base_clf_option", BASE_CLASSIFIERS)
    best_classifier = best_base_classifier(base_clf_option)
    classifier_obj = BaggingPuClassifier(
        base_estimator=best_classifier,
        n_estimators=trial.suggest_int("num_bagged", 25, 200),
        max_samples=trial.suggest_float("max_samples", 0.1, 1.0),
        max_features=trial.suggest_float("max_features", 0.1, 1.0),
    )
    classifier_obj.fit(X_train_scaled, y_train)
    y_pred = classifier_obj.predict(X_val_scaled)
    return lee_liu_score(y_known=y_val, y_pred=y_pred)


sampler = TPESampler(seed=RANDOM_SEED)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(bagging_objective, n_trials=NUM_TRIALS)

In [ ]:
best_params = study.best_params
print(f"{best_params=}")
optimised_bagging = BaggingPuClassifier(
    base_estimator=best_base_classifier(best_params["base_clf_option"]),
    n_estimators=best_params["num_bagged"],
    max_samples=best_params["max_samples"],
    max_features=best_params["max_features"],
    random_state=RANDOM_SEED,
)
optimised_models.append(optimised_bagging)
data = {
    "model": "Bagging Classifier",
    "params": best_params,
    "ll_score": study.best_value,
}
models_info.append(data)
print(data)

### Modified Logistic Regression


In [ ]:
def mlr_objective(trial):
    classifier_obj = ModifiedLogisticRegression(
        epochs=250,
        learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-1),
    )
    classifier_obj.fit(X_train_scaled, y_train)
    y_pred = classifier_obj.predict(X_val_scaled)
    return lee_liu_score(y_known=y_val, y_pred=y_pred)


sampler = TPESampler(seed=RANDOM_SEED)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(mlr_objective, n_trials=NUM_TRIALS)

best_params = study.best_params
print(f"{best_params=}")
optimised_mlr = ModifiedLogisticRegression(epochs=250, **best_params)
optimised_models.append(optimised_mlr)
data = {"model": "MLR", "params": best_params, "ll_score": study.best_value}
models_info.append(data)
print(data)

### PUExtraTrees

In [ ]:
from utils import c_to_alpha

np.random.seed(RANDOM_SEED)


def puet_objective(trial):
    np.random.seed(RANDOM_SEED)
    classifier_obj = PUExtraTrees(
        n_estimators=trial.suggest_int("n_estimators", 25, 200),
        risk_estimator=trial.suggest_categorical("risk_estimator", ["nnPU", "uPU"]),
        loss=trial.suggest_categorical("loss", ["quadratic", "logistic"]),
        min_samples_leaf=trial.suggest_int("min_samples_leaf", 1, 10),
        max_features=trial.suggest_categorical("max_features", ["sqrt", "all"]),
        max_candidates=trial.suggest_int("max_candidates", 1, 10),
    )

    optimised_elkanoto.fit(X_train_scaled, y_train)
    elkan_c = optimised_elkanoto.c
    elkan_alpha = c_to_alpha(y, elkan_c)

    classifier_obj.fit(X_train_scaled, y_train, alpha=elkan_alpha)
    y_pred = classifier_obj.predict(X_val_scaled)
    return lee_liu_score(y_known=y_val, y_pred=y_pred)


sampler = TPESampler(seed=RANDOM_SEED)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(puet_objective, n_trials=NUM_TRIALS)

best_params = study.best_params
optimised = PUExtraTrees(**best_params)
optimised_models.append(optimised)
data = {"model": "PUExtraTrees", "params": best_params, "ll_score": study.best_value}
models_info.append(data)
print(data)

In [ ]:
# Look at the LL scores of the optimised models on this particular train-test split.
# This is not likely to be a fair representation of the models
models_df = pd.DataFrame(models_info)
models_df.sort_values("ll_score", ascending=False)

# Repeater Candidate Identification

In [ ]:
np.random.seed(RANDOM_SEED)
NUM_ITERATIONS = 1000
MIN_VOTES_FOR_REPEATER_CANDIDATE = 3

models_fit_predict_times = [[] for _ in range(len(optimised_models))]
models_recalls = [[] for _ in range(len(optimised_models))]
models_llscores = [[] for _ in range(len(optimised_models))]
models_frac_pos = [[] for _ in range(len(optimised_models))]
models_priors = [[] for _ in range(len(optimised_models))]

candidates = {}

# Save model and SHAP values from the last iteration
explainers = []
models_shap_values = []

for j in range(NUM_ITERATIONS):
    if j % (NUM_ITERATIONS * 0.05) == 0:
        print(f"Trial {j} / {NUM_ITERATIONS}")

    # Get training data for this iteration and rescale it
    X_train, _, y_train, _ = get_subburst_preserved_train_test(
        original_df=df, X=X, y=y, test_size=0.2
    )
    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_scaled = scaler.transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns=FEATURES)

    models_predictions = []
    round_priors = []

    for i, model in enumerate(optimised_models):
        model_name = model.__class__.__name__

        start = time.time()
        if model_name == "PUExtraTrees":
            puet_c_val = optimised_elkanoto.c
            model.fit(X=X_train_scaled, y=y_train, alpha=c_to_alpha(y, puet_c_val))
        else:
            model.fit(X_train_scaled, y_train)

        # Save metrics and evaluation score
        predictions = model.predict(X_scaled)
        models_predictions.append(predictions)
        models_fit_predict_times[i].append(time.time() - start)
        models_recalls[i].append(recall_score(y, predictions))
        models_llscores[i].append(lee_liu_score(y, predictions))
        frac_pos = sum(predictions == 1) / len(predictions)
        models_frac_pos[i].append(frac_pos)

        # Save the prior for models which estiamte it
        if model_name in [
            "ElkanotoPuClassifier",
            "WeightedElkanotoPuClassifier",
            "ModifiedLogisticRegression",
            "PUExtraTrees",
        ]:
            if (
                model_name == "ElkanotoPuClassifier"
                or model_name == "WeightedElkanotoPuClassifier"
            ):
                c = model.c
                prior = c_to_alpha(y_train, c)
            elif model_name == "ModifiedLogisticRegression":
                c = model.c_hat[0]
                prior = c_to_alpha(y_train, c)
            elif model_name == "PUExtraTrees":
                prior = c_to_alpha(y_train, puet_c_val)
            models_priors[i].append(prior)

        # Save SHAP values for each model in the last iteration
        if j == NUM_ITERATIONS - 1:
            explainer = shap.Explainer(model.predict, X_scaled)
            explainer.feature_names = FEATURE_LABELS
            explainers.append(explainer)
            shap_values = explainer(X_scaled)
            models_shap_values.append(shap_values)

    # Get predictions on the full dataset
    avg_preds = np.mean(models_predictions, axis=0)
    flagged = np.where(
        avg_preds > (MIN_VOTES_FOR_REPEATER_CANDIDATE / len(optimised_models))
    )[0]

    # Loop through the flagged indices and identify repeater candidates
    for index in flagged:
        if df.iloc[index]["is_repeater"] == 1:
            continue
        tns_name = df.iloc[index]["tns_name"]
        sub_num = df.iloc[index]["sub_num"]
        key = f"{tns_name}_{sub_num}"
        if key not in candidates:
            candidates[key] = 0
        candidates[key] += 1

## Compare models

In [ ]:
models_df["mean_fit_time"] = [
    np.mean(fittimes) for fittimes in models_fit_predict_times
]
models_df["std_fit_time"] = [np.std(fittimes) for fittimes in models_fit_predict_times]
models_df["mean_recall"] = [np.mean(recalls) for recalls in models_recalls]
models_df["std_recall"] = [np.std(recalls) for recalls in models_recalls]
models_df["mean_ll_score"] = [np.mean(ll_scores) for ll_scores in models_llscores]
models_df["std_ll_score"] = [np.std(ll_scores) for ll_scores in models_llscores]
models_df["frac_pos"] = [np.mean(frac_pos) for frac_pos in models_frac_pos]
models_df["std_frac_pos"] = [np.std(frac_pos) for frac_pos in models_frac_pos]
models_df["mean_prior"] = [np.mean(priors) for priors in models_priors]
models_df["std_prior"] = [np.std(priors) for priors in models_priors]

models_df.sort_values(by="mean_ll_score", ascending=False)

In [ ]:
models_df.sort_values(by="mean_ll_score", ascending=False)[
    ["model", "mean_recall", "std_recall", "mean_ll_score", "std_ll_score"]
].to_latex("tables/puml_models_optimised.tex", index=False)

## Extract candidates

In [ ]:
# A sub-burst must be flagged in 10% of iterations to be considered a repeater candidate
MIN_ITERS_REPEATER_CANDIDATE = 0.1 * NUM_ITERATIONS

# Process the candidates into a dataframe
candidates_df = pd.DataFrame.from_dict(candidates, columns=["count"], orient="index")
candidates_df["tns_name"] = candidates_df.index
candidates_df = candidates_df[["tns_name", "count"]]
candidates_df.reset_index(drop=True, inplace=True)

for index, row in candidates_df.iterrows():
    # Convert the underscore format of the tns_name back to a split tns_name and sub_num
    if row["tns_name"][-2] == "_":
        tns_name = row["tns_name"][:-2]
        sub_num = int(row["tns_name"][-1:])
        candidates_df.loc[index, "tns_name"] = tns_name
        candidates_df.loc[index, "sub_num"] = int(sub_num)

# Filter out candidates that were not flagged in the required fraction of predictions
candidates_df.to_csv("./data/candidates/all_candidates.csv", index=False)
print(f"Number of sub-bursts flagged at least once: {len(candidates_df)}")
candidates_df = candidates_df[candidates_df["count"] >= (MIN_ITERS_REPEATER_CANDIDATE)]
print(f"Number of sub-burst candidates with >=10% confidence: {len(candidates_df)}")
candidates_df.sort_values(by="count", ascending=False)

In [ ]:
print("Total sources identified:", candidates_df["tns_name"].nunique())

In [ ]:
# Set initial column values
df["confidence"] = 1
df["is_candidate"] = 0
df["is_ref_candidate"] = 0
df["overlapping_papers"] = ""
df["num_overlapping"] = 0

# Loop through the candidates df and update the original dataframe with the right flags
for index, row in candidates_df.iterrows():
    tns_name = row["tns_name"]
    sub_num = row["sub_num"]
    original_row = df[(df["tns_name"] == tns_name) & (df["sub_num"] == sub_num)]

    df.loc[original_row.index, "is_candidate"] = 1
    df.loc[original_row.index, "confidence"] = row["count"] / NUM_ITERATIONS

# Note whether that sub-burst was flagged by previous ML papers as a repeater candidate
for index, row in df.iterrows():
    tns_name = row["tns_name"]
    if tns_name in REFERENCE_PAPER_REPEATERS:
        df.loc[index, "is_ref_candidate"] = 1

    # Now add a string indicating which papers it overlapped with
    results = []
    if tns_name in MORPHOLOGY_REPEATERS:  # Pleunis et al. 2021
        results.append(1)
    if tns_name in SUPERVISED_PAPER_REPEATERS:  # Luo et al. 2022
        results.append(2)
    if tns_name in UNSUPERVISED_PAPER_REPEATERS:  # Zhu-Ge et al. 20222
        results.append(3)
    result_str = "".join(
        [
            f"{res}, " if i < (len(results) - 1) else str(res)
            for i, res in enumerate(results)
        ]
    )
    df.loc[index, "overlapping_papers"] = result_str
    df.loc[index, "num_overlapping"] = len(results)

In [ ]:
# Export particular columns to a latex table for appendix

df["count"] = df["confidence"] * NUM_ITERATIONS
export_df = df[df["is_candidate"] == 1].sort_values(
            ["count", "num_overlapping", "tns_name", "sub_num", "is_pcc_candidate"],
            ascending=[False, False, False, False, True]
)[["tns_name", "sub_num"] + ["count", "is_pcc_candidate", "overlapping_papers"]]
export_df["count"] = export_df["count"].astype(int)
export_df["is_pcc_candidate"] = export_df["is_pcc_candidate"].apply(
    lambda x: r"\checkmark" if x == 1 else r"\xmark"
)
export_df.columns = ["TNS Name", "Sub-burst"] + ["Count", "From silver sample", "Overlapping with"]
export_df.to_latex("tables/candidates.tex", index=False)

In [ ]:
df[df["is_candidate"] == 1]["count"].describe()

# Feature importance

We use the SHAP explainable AI library to evaluate feature importance

## Overall

In [ ]:
def avg_pred_function(row):
    """
    Get predictions from the 1000th iteration of all the classifiers, and average them. Use this for SHAP analysis.
    """
    preds = []
    # These are the same as the model objects from the last round of training. Therefore, the predictions will be the same as in the last round
    for model in optimised_models:
        preds.append(model.predict(row))
    mean_preds = np.mean(preds, axis=0)
    return [round(x) for x in mean_preds]


avg_explainer = shap.Explainer(avg_pred_function, X_scaled, random_state=RANDOM_SEED)
avg_explainer.feature_names = FEATURE_LABELS
avg_shap_values = avg_explainer(X_scaled)

In [ ]:
plt.clf()
shap.plots.beeswarm(
    shap_values=avg_shap_values,
    max_display=len(FEATURES),
    color=custom_cmap,
    show=False,
)
plt.xlabel("SHAP value")
plt.ylabel("Feature", fontsize=14)
plt.tight_layout()
plt.savefig("./figures/puml/beeswarm_averaged.png", bbox_inches="tight", dpi=300)
plt.show()

## FI for each model

In [ ]:
# Get the mean SHAP importance values for each feature across all models
exparr = np.array([abs(shap_values.values) for shap_values in models_shap_values])
mean_importances = np.mean(np.mean(exparr, axis=1), axis=0)

# Get the mean SHAP importance values for each feature for each model, to be plotted on a stacked bar chart
importances_info = []
for i, model_name in enumerate(["CE", "WE", "BC", "MLR", "PUET"]):
    importances = np.mean(abs(models_shap_values[i].values), axis=0)
    for i, (name, importance) in enumerate(zip(FEATURE_LABELS, importances)):
        importances_info.append(
            {
                "Classifier": model_name,
                "Feature": name,
                "Importance": importance,
                "Mean importance": mean_importances[i],
            }
        )
# Sorting by mean importance means that the same features will be grouped together
importances_info = pd.DataFrame(importances_info).sort_values(
    "Mean importance", ascending=False
)

sns.set_palette("colorblind")
plt.figure(figsize=(3, 6))  # increase the height of the plot so that bars are thicker
ax = sns.barplot(
    data=importances_info, x="Importance", y="Feature", hue="Classifier", linewidth=2.5
)
plt.xlabel("Importance (mean SHAP value)", fontsize=12)
plt.ylabel("Feature", fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

# Add mean importance to next to each stacked bar plot
ax.set_xlim(0, 0.24)
for i, imp in enumerate(sorted(mean_importances, reverse=True)):
    plt.text(0.21, i, round(imp, 3), ha="center", va="center")

# Add light borders between each feature
for i in range(len(FEATURES)):
    plt.axhline(i + 0.5, color="lightgray", linewidth=1)

plt.savefig("./figures/puml/all_feature_importances.png", bbox_inches="tight")

# Candidate Analysis

In [ ]:
np.random.seed(RANDOM_SEED)

# Resort so that the correct bursts are at the top in the plots
df.sort_values(
    by=["is_candidate", "is_ref_candidate", "is_pcc_candidate", "is_repeater"],
    inplace=True,
)
X = df[FEATURES]

# Rescale and transform the entire dataset for visualisation in UMAP space
X_scaled = StandardScaler().fit_transform(X)
X_embedded = UMAP(random_state=RANDOM_SEED, n_neighbors=20).fit_transform(X_scaled)
sns.set_palette("colorblind")

In [ ]:
df21 = pd.read_csv('./data/raw_data/chime_frb_catalog_2021.csv')
tns_21 = list(df21['tns_name'])
df['is_2021'] = df.apply(lambda x: True if x['tns_name'] in tns_21 else False, axis=1)

In [ ]:
LINE_WIDTH = 0.375

In [ ]:
# Plot candidates on scatterplot
plt.clf()
plot = sns.scatterplot(
    x=X_embedded[:, 0],
    y=X_embedded[:, 1],
    hue=df.apply(
        lambda row: 0
        if row["is_candidate"] == 1
        else 1
        if row["is_repeater"] == 1
        else 2,
        axis=1,
    ),
    style=df.apply(
        lambda row: 0
        if row["is_candidate"] == 1
        else 1
        if row["is_repeater"] == 1
        else 2,
        axis=1,
    ),
    markers={0: "X", 1: "o", 2: "o"},
    palette={0: "#0173b2", 1: "#cc78bc", 2: "#029e73"},
    edgecolor="black",
    linewidth=LINE_WIDTH,
)

handles, _ = plot.get_legend_handles_labels()
plot.legend(
    handles=handles,
    labels=["Candidate", "Known repeater", "Non-repeater"],
    loc=(0, -0.2),
)
plt.axis("off")
plt.savefig(
    f"./figures/puml/visualisations/simple-UMAP-candidates.png",
    dpi=500,
    bbox_inches="tight",
)
plt.show()

## Comparison with previous work

Below, we compare the candidates we flagged with the results of [Luo et al (2022)](https://arxiv.org/abs/2210.02463), [Zhu-Ge et al. (2023)](https://arxiv.org/abs/2210.02471), and [Pleunis et al (2021)](https://iopscience.iop.org/article/10.3847/1538-4357/ac33ac)

In [ ]:
# These list the number of sub-bursts, i.e, the points on the plot
df.apply(
    lambda row: 0
    if row["is_candidate"] == 1 and row["is_ref_candidate"] == 1
    else 1
    if row["is_candidate"] == 1
    else 2
    if row["is_ref_candidate"] == 1
    else 3
    if row["is_repeater"] == 1
    else 4,
    axis=1,
).value_counts()

In [ ]:
# Filter by sub-bursts which the previous papers had available to them
df[df.apply(lambda row: '202' not in row["tns_name"], axis=1)].apply(
    lambda row: 0 if row["is_candidate"] == 1 and row["is_ref_candidate"] == 1
    else 1 if row["is_candidate"] == 1
    else 2 if row["is_ref_candidate"] == 1
    else 3,
    axis=1,
).value_counts()

In [ ]:
# These are the bursts which we identify to be new repeating sources that the previous papers did not
new_sources = df[
    (df["is_ref_candidate"] == 0)
    & (df["is_candidate"] == 1)
    & (df.apply(lambda row: "202" not in row["tns_name"], axis=1))
]["tns_name"].unique()
print(
    f"{len(new_sources)} new sources not identified by reference papers:", new_sources
)

In [ ]:
# These are the bursts which they identify to be repeating sources that we do not 
missed_sources = df[
    (df["is_ref_candidate"] == 1)
    & (df["is_candidate"] == 0)
]["tns_name"].unique()
print(
    f"{len(missed_sources)} sources identified by reference papers but not by us:", missed_sources
)

In [ ]:
# Constants for palette
PALETTE_HEX = {0: "#0173b2", 1: "#029e73", 2: "#fbafe4", 3: "#de8f05", 4: "#949494"}
EXPANDED_PALETTE_RGBA = {0: (1/255, 115/255, 178/255, 1.0),
        1: (2/255, 158/255, 115/255, 1.0),
        2: (251/255, 175/255, 228/255, 1.0),
        3: (222/255, 143/255, 5/255, 1.0),
        4: (148/255, 148/255, 148/255, 1.0),
        5: (1/255, 115/255, 178/255, 0.4),
        6: (2/255, 158/255, 115/255, 0.4),
        7: (251/255, 175/255, 228/255, 0.4),
        8: (222/255, 143/255, 5/255, 0.4),
        9: (148/255, 148/255, 148/255, 0.4),
}
EXPANDED_EDGE_COLORS = {0: "black", 1: "black", 2: "black", 3: "black", 4: "black", 5: "none", 6: "none", 7: "none", 8: "none", 9: "none"}
MARKERS = {0: "X", 1: "D", 2: "v", 3: "o", 4: "*"}
ICON_SIZE_STD = 50
LEGEND_LOC = (0, -0.3)

def gen_legend_handles(labels: list[str], markers=MARKERS, palette:dict=PALETTE_HEX, markersize=ICON_SIZE_STD * 0.15):
        """
        Generate legend handles for a scatterplot
        """
        return [
                Line2D(
                [0],
                [0],
                marker=markers[i],
                color=PALETTE_HEX[i],
                markerfacecolor=palette[i],
                markersize=markersize,
                linestyle="None",
                label=label,
                )
                for i, label in enumerate(labels)
        ]

In [ ]:
plt.clf()

hue_style_values = df.apply(
        lambda row: 0 if row["is_candidate"] == 1 and row["is_ref_candidate"] == 1 and row["is_2021"] == 1 
        else 1 if row["is_candidate"] == 1 and row["is_2021"] == 1  
        else 2 if row["is_ref_candidate"] == 1 and row["is_2021"] == 1 
        else 3 if row["is_repeater"] == 1 and row["is_2021"] == 1 
        else 4 if row["is_2021"] == 1

        else 5 if row["is_candidate"] == 1 and row["is_ref_candidate"] == 1 and row["is_2021"] == 0
        else 6 if row["is_candidate"] == 1 and row["is_2021"] == 0
        else 7 if row["is_ref_candidate"] == 1 and row["is_2021"] == 0
        else 8 if row["is_repeater"] == 1 and row["is_2021"] == 0
        else 9,
        axis=1,
)
style_values = df.apply(
        lambda row: 0 if row["is_candidate"] == 1 and row["is_ref_candidate"] == 1
        else 1 if row["is_candidate"] == 1
        else 2 if row["is_ref_candidate"] == 1
        else 3 if row["is_repeater"] == 1
        else 4,
        axis=1,
)
plot = sns.scatterplot(
    x=X_embedded[:, 0],
    y=X_embedded[:, 1],

    style=style_values,
    markers=MARKERS,
    linewidth=LINE_WIDTH,

    hue=hue_style_values,
    palette=EXPANDED_PALETTE_RGBA,
    edgecolor=hue_style_values.map(EXPANDED_EDGE_COLORS),

    # Adjust for a visual oddity that results in the stars without borders appearing too small
    size=(df.apply(lambda row: ICON_SIZE_STD*1.6 if (row["is_repeater"] == False and row["is_2021"] == False and row["is_candidate"] == False) else ICON_SIZE_STD, axis=1)),
    sizes=(ICON_SIZE_STD, ICON_SIZE_STD*1.6),

)
handles, _  = plot.get_legend_handles_labels()
plot.legend(
    handles=gen_legend_handles(labels=[
        "Overlapping candidates",
        "New candidates with PU learning",
        "Candidates from previous work",
        "Known repeaters",
        "Non-repeaters",
    ]),
    loc=LEGEND_LOC
)
plt.axis("off")
plt.savefig(
    f"./figures/puml/visualisations/UMAP-candidate_compare.png",
    dpi=500,
    bbox_inches="tight",
)
plt.show()


### Comparison with silver sample

We can analyse the model's predictions on the 14 'silver sample' candidates identified by [CHIME/FRB 2023](https://iopscience.iop.org/article/10.3847/1538-4357/acc6c1)

In [ ]:
plt.clf()

hue_style_values = df.apply(
        lambda row: 0 if row["is_candidate"] == 1 and row["is_pcc_candidate"] == 1 and row["is_2021"] == 0 
        else 1 if row["is_candidate"] == 1 and row["is_2021"] == 0  
        else 2 if row["is_pcc_candidate"] == 1 and row["is_2021"] == 0 
        else 3 if row["is_repeater"] == 1 and row["is_2021"] == 0 
        else 4 if row["is_2021"] == 0

        else 5 if row["is_candidate"] == 1 and row["is_pcc_candidate"] == 1 and row["is_2021"] == 1
        else 6 if row["is_candidate"] == 1 and row["is_2021"] == 1
        else 7 if row["is_pcc_candidate"] == 1 and row["is_2021"] == 1
        else 8 if row["is_repeater"] == 1 and row["is_2021"] == 1
        else 9,
        axis=1,
)
style_values = df.apply(
        lambda row: 0 if row["is_candidate"] == 1 and row["is_pcc_candidate"] == 1
        else 1 if row["is_candidate"] == 1
        else 2 if row["is_pcc_candidate"] == 1
        else 3 if row["is_repeater"] == 1
        else 4,
        axis=1,
    )
plot = sns.scatterplot(
    x=X_embedded[:, 0],
    y=X_embedded[:, 1],

    style=style_values,
    markers=MARKERS,
    linewidth=LINE_WIDTH,

    hue=hue_style_values,
    palette=EXPANDED_PALETTE_RGBA,
    edgecolor=hue_style_values.map(EXPANDED_EDGE_COLORS),
)
plt.legend(handles=gen_legend_handles(labels=[
                "Overlapping candidates",
                "New candidates with PU learning",
                "Silver sample candidate",
                "Known repeaters",
                "Non-repeaters"
           ]),
           loc=LEGEND_LOC,
)
plt.axis("off")
plt.savefig(    
    f"./figures/puml/visualisations/UMAP-2023_chime_candidates.png",
    dpi=500,
    bbox_inches="tight",
)
plt.show()

Below, we compare our confidence to the $R_{CC}$ value. We should ideally see an inverse correlation.

In [ ]:
chime_frb_2023_silver = pd.read_csv("./data/candidates/2023_chimefrb_silver.csv")
all_candidates_df = pd.read_csv("./data/candidates/all_candidates.csv")

for index, row in chime_frb_2023_silver.iterrows():
    # Check if we identified the exact same sub-burst
    tns_name = row["tns_name"]
    sub_num = row["sub_num"]
    try:
        df_row = all_candidates_df[
            (all_candidates_df["tns_name"] == tns_name)
            & (all_candidates_df["sub_num"] == sub_num)
        ]
        confidence = df_row["count"].values[0] / NUM_ITERATIONS
    except Exception as e:
        # Not identified as candidate
        confidence = 0

    chime_frb_2023_silver.loc[index, "confidence"] = confidence
chime_frb_2023_silver["R_cc_recip"] = 1 / chime_frb_2023_silver["R_cc"]
chime_frb_2023_silver[
    ["tns_name", "repeater_name", "confidence", "R_cc", "R_cc_recip"]
].sort_values(["confidence", "R_cc"])

In [ ]:
chime_frb_2023_silver[chime_frb_2023_silver["confidence"] > 0][
    "repeater_name"
].nunique()

A more generous approach would be to assume that, if we identify _any_ sub-burst in a cluster from the silver sample, we have identified the entire silver sample.

In [ ]:
for index, row in chime_frb_2023_silver.iterrows():
    repeater_name = row["repeater_name"]

    # Check if the first burst in the cluster is identified as a candidate
    if repeater_name in list(all_candidates_df["tns_name"]):
        chime_frb_2023_silver.loc[index, "confidence"] = (
            all_candidates_df[all_candidates_df["tns_name"] == repeater_name][
                "count"
            ].values[0]
            / NUM_ITERATIONS
        )

    # Find all the other rows with the same repeater name in the silver sample
    other_rows = chime_frb_2023_silver[
        chime_frb_2023_silver["repeater_name"] == repeater_name
    ]
    for other_idx, other_row in other_rows.iterrows():
        tns_name = other_row["tns_name"]
        sub_num = other_row["sub_num"]
        if tns_name in list(all_candidates_df["tns_name"]):
            chime_frb_2023_silver.loc[index, "confidence"] = (
                all_candidates_df[all_candidates_df["tns_name"] == tns_name][
                    "count"
                ].values[0]
                / NUM_ITERATIONS
            )

chime_frb_2023_silver[
    ["tns_name", "repeater_name", "confidence", "R_cc", "R_cc_recip"]
].sort_values(
    [
        "confidence",
        "repeater_name",
    ]
)

In [ ]:
silver_overlapping_candidates = chime_frb_2023_silver[
    chime_frb_2023_silver["confidence"] > 0
]

In [ ]:
# Now update the original df again
copied_df = df.copy()
for index, row in copied_df.iterrows():
    if row["tns_name"] in silver_overlapping_candidates["tns_name"].values:
        if row["is_candidate"] != 1:
            copied_df.loc[index, "is_candidate"] = 1
print("\nOld:")
print(
    df.apply(
        lambda row: 0
        if row["is_candidate"] == 1 and row["is_pcc_candidate"] == 1
        else 1
        if row["is_candidate"] == 1
        else 2
        if row["is_pcc_candidate"] == 1
        else 3
        if row["is_repeater"] == 1
        else 4,
        axis=1,
    ).value_counts()
)
print("\nNew:")
print(
    copied_df.apply(
        lambda row: 0
        if row["is_candidate"] == 1 and row["is_pcc_candidate"] == 1
        else 1
        if row["is_candidate"] == 1
        else 2
        if row["is_pcc_candidate"] == 1
        else 3
        if row["is_repeater"] == 1
        else 4,
        axis=1,
    ).value_counts()
)

With this perspective, we've identified 18, not 10, overlapping sub-bursts. Below, we confirm below that the repeater candidates identified are the same

In [ ]:
silver_overlapping_candidates["repeater_name"].nunique()

In [ ]:
plt.clf()

hue_style_values = copied_df.apply(
        lambda row: 0 if row["is_candidate"] == 1 and row["is_pcc_candidate"] == 1 and row["is_2021"] == 0 
        else 1 if row["is_candidate"] == 1 and row["is_2021"] == 0  
        else 2 if row["is_pcc_candidate"] == 1 and row["is_2021"] == 0 
        else 3 if row["is_repeater"] == 1 and row["is_2021"] == 0 
        else 4 if row["is_2021"] == 0

        else 5 if row["is_candidate"] == 1 and row["is_pcc_candidate"] == 1 and row["is_2021"] == 1
        else 6 if row["is_candidate"] == 1 and row["is_2021"] == 1
        else 7 if row["is_pcc_candidate"] == 1 and row["is_2021"] == 1
        else 8 if row["is_repeater"] == 1 and row["is_2021"] == 1
        else 9,
        axis=1,
)
style_values = copied_df.apply(
        lambda row: 0 if row["is_candidate"] == 1 and row["is_pcc_candidate"] == 1
        else 1 if row["is_candidate"] == 1
        else 2 if row["is_pcc_candidate"] == 1
        else 3 if row["is_repeater"] == 1
        else 4,
        axis=1,
    )
plot = sns.scatterplot(
    x=X_embedded[:, 0],
    y=X_embedded[:, 1],

    style=style_values,
    markers=MARKERS,
    linewidth=LINE_WIDTH,

    hue=hue_style_values,
    palette=EXPANDED_PALETTE_RGBA,
    edgecolor=hue_style_values.map(EXPANDED_EDGE_COLORS),
)
plt.legend(handles=gen_legend_handles(labels=[
                "Overlapping candidates",
                "New candidates with PU learning",
                "Silver sample candidate",
                "Known repeaters",
                "Non-repeaters"
           ]),
           loc=LEGEND_LOC,
)
plt.axis("off")
plt.savefig(
    f"./figures/puml/visualisations/UMAP-2023_chime_candidates-all.png",
    dpi=500,
    bbox_inches="tight",
)
plt.show()

We try to better understand why the results are so different, by comparing the candidate repeaters and the silver sample repeaters using the k-sample Anderson-Darling test.

In [ ]:
from scipy.stats import anderson_ksamp

pvals = []

for i, feature in enumerate(FEATURES):
    res = anderson_ksamp(
        samples=[
            df[df["is_repeater"] == 1][feature],
            df[df["is_pcc_candidate"] == 1][feature],
        ]
    )
    rep_pcc_stat = res.statistic
    rep_pcc_p_value = res.significance_level

    res = anderson_ksamp(
        [df[df["is_repeater"] == 1][feature], df[df["is_candidate"] == 1][feature]]
    )
    rep_cand_stat = res.statistic
    rep_cand_p_value = res.significance_level

    res = anderson_ksamp(
        [
            df[df["is_repeater"] == 1][feature],
            df[(df["is_pcc_candidate"] == 1) & (df["is_candidate"] == 0)][feature],
        ]
    )
    rep_pcc_not_cand_stat = res.statistic
    rep_pcc_not_cand_p_value = res.significance_level

    pvals.append(
        {
            "feature": FEATURE_LABELS[i],
            "Candidates": rep_cand_p_value,
            "RCC": rep_pcc_p_value,
            "RCC Non-candidates": rep_pcc_not_cand_p_value,
        }
    )
pd.options.display.float_format = '{:.10f}'.format
pd.DataFrame(pvals).to_latex("tables/ad_pvals.tex", index=False)
pd.DataFrame(pvals).head(20)